In [1]:
plotHeight <- 6
options(repr.plot.width=10, repr.plot.height=plotHeight)

In [2]:
library(lme4)
library(tidyverse)

Loading required package: Matrix
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [3]:
incidents <- read.csv('../data/processed/master-incidents.csv', stringsAsFactors=FALSE) %>% 
    mutate(Date = as.Date(BeginDateTime))
demographics <- read.csv('../data/processed/neighborhood-demographics.csv', stringsAsFactors=FALSE)
weather <- read.csv('../data/processed/cleaned-weather.csv', stringsAsFactors=FALSE) %>% 
    mutate(Date = as.Date(Date))
head(weather)

Date,High,Low,Precipitation,SnowPrecipitation,SnowDepth,IsPrecipitation,IsSnowPrecipitation,IsGroundSnow
<date>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>
2010-01-01,6,-9,0,0,9,1,1,1
2010-01-02,1,-15,0,0,9,0,0,1
2010-01-03,7,-14,0,0,9,0,0,1
2010-01-04,7,-10,0,0,9,0,0,1
2010-01-05,10,-9,0,0,9,0,0,1
2010-01-06,16,-4,0,0,9,0,0,1


In [4]:
dailyIncidents <- incidents %>% 
    count(Date) %>% 
    rename(Incidents=n)
head(dailyIncidents)

Date,Incidents
<date>,<int>
2010-01-01,62
2010-01-02,38
2010-01-03,37
2010-01-04,62
2010-01-05,44
2010-01-06,37


In [5]:
summary(glm(Incidents~1, family=poisson, data=dailyIncidents))


Call:
glm(formula = Incidents ~ 1, family = poisson, data = dailyIncidents)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-7.7543  -1.2987  -0.0751   1.2112   6.5507  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) 4.052979   0.002221    1825   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 11630  on 3521  degrees of freedom
Residual deviance: 11630  on 3521  degrees of freedom
AIC: 32285

Number of Fisher Scoring iterations: 4


In [6]:
dates <- unique(incidents$Date)
neighborhoods <- unique(incidents$Neighborhood)
dateNeighborhood <- expand.grid(Date=dates, Neighborhood=neighborhoods)

dailyNeighborhoodIncidents <- incidents %>% 
    count(Date, Neighborhood) %>% 
    rename(Incidents = n) %>% 
    right_join(dateNeighborhood, c('Date', 'Neighborhood')) %>% 
    mutate(Incidents = replace_na(Incidents, 0)) %>% 
    inner_join(demographics, 'Neighborhood') %>% 
    left_join(weather, 'Date') %>% 
    rename(Temperature=High)

head(dailyNeighborhoodIncidents)

Warning message:
“Column `Neighborhood` joining character vector and factor, coercing into character vector”

Date,Neighborhood,Incidents,Population,AgeU5,Age5To9,Age10To14,Age15To17,Age18To19,Age20To24,⋯,Colored,Hispanic,Temperature,Low,Precipitation,SnowPrecipitation,SnowDepth,IsPrecipitation,IsSnowPrecipitation,IsGroundSnow
<date>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>
2018-06-30,WILLARD - HAY,1,8611,739,799,847,564,338,689,⋯,6530,643,89,75,0.00,0,0,1,0,0
2018-06-29,WILLARD - HAY,1,8611,739,799,847,564,338,689,⋯,6530,643,99,74,0.00,0,0,0,0,0
2018-06-22,WILLARD - HAY,0,8611,739,799,847,564,338,689,⋯,6530,643,82,63,0.00,0,0,0,0,0
2018-06-27,WILLARD - HAY,1,8611,739,799,847,564,338,689,⋯,6530,643,82,65,0.00,0,0,0,0,0
2018-07-01,WILLARD - HAY,2,8611,739,799,847,564,338,689,⋯,6530,643,75,63,1.16,0,0,1,0,0
2018-06-26,WILLARD - HAY,0,8611,739,799,847,564,338,689,⋯,6530,643,73,64,1.10,0,0,1,0,0


In [7]:
summary(glm(Incidents~1, family=poisson, data=dailyNeighborhoodIncidents))


Call:
glm(formula = Incidents ~ 1, family = poisson, data = dailyNeighborhoodIncidents)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1604  -1.1604  -1.1604   0.3712  13.5822  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.395647   0.002241  -176.6   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 456797  on 295847  degrees of freedom
Residual deviance: 456797  on 295847  degrees of freedom
AIC: 727504

Number of Fisher Scoring iterations: 6


In [8]:
summary(glm(Incidents~Neighborhood-1, family=poisson, data=dailyNeighborhoodIncidents, offset=log(Population)))


Call:
glm(formula = Incidents ~ Neighborhood - 1, family = poisson, 
    data = dailyNeighborhoodIncidents, offset = log(Population))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.5954  -0.8942  -0.6368   0.4474  14.5885  

Coefficients:
                                               Estimate Std. Error z value
NeighborhoodARMATAGE                         -10.052784   0.036835  -272.9
NeighborhoodAUDUBON PARK                      -9.342672   0.025557  -365.6
NeighborhoodBANCROFT                          -9.224136   0.029223  -315.6
NeighborhoodBELTRAMI                          -8.941712   0.041703  -214.4
NeighborhoodBOTTINEAU                         -8.819689   0.034060  -258.9
NeighborhoodBRYANT                            -9.309046   0.033260  -279.9
NeighborhoodBRYN - MAWR                       -9.285592   0.033981  -273.3
NeighborhoodCEDAR - ISLES - DEAN              -9.090174   0.029476  -308.4
NeighborhoodCEDAR RIVERSIDE                   -9.055235   0.0

In [9]:
summary(glm(Incidents~Neighborhood+Temperature-1, family=poisson, data=dailyNeighborhoodIncidents, 
           offset=log(Population)))


Call:
glm(formula = Incidents ~ Neighborhood + Temperature - 1, family = poisson, 
    data = dailyNeighborhoodIncidents, offset = log(Population))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.9409  -0.8720  -0.6319   0.4617  15.1980  

Coefficients:
                                               Estimate Std. Error z value
NeighborhoodARMATAGE                         -1.039e+01  3.727e-02 -278.85
NeighborhoodAUDUBON PARK                     -9.683e+00  2.618e-02 -369.87
NeighborhoodBANCROFT                         -9.564e+00  2.977e-02 -321.29
NeighborhoodBELTRAMI                         -9.282e+00  4.209e-02 -220.54
NeighborhoodBOTTINEAU                        -9.160e+00  3.453e-02 -265.27
NeighborhoodBRYANT                           -9.649e+00  3.374e-02 -285.99
NeighborhoodBRYN - MAWR                      -9.626e+00  3.445e-02 -279.39
NeighborhoodCEDAR - ISLES - DEAN             -9.430e+00  3.002e-02 -314.17
NeighborhoodCEDAR RIVERSIDE                  -9

In [10]:
mixedEffectsModel <- glmer(Incidents~Temperature + (1 | Neighborhood), family=poisson, 
                      data=dailyNeighborhoodIncidents, offset=log(Population))
summary(mixedEffectsModel)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?”

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: poisson  ( log )
Formula: Incidents ~ Temperature + (1 | Neighborhood)
   Data: dailyNeighborhoodIncidents
 Offset: log(Population)

      AIC       BIC    logLik  deviance  df.resid 
 568246.4  568278.2 -284120.2  568240.4    295845 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-2.786 -0.621 -0.452  0.501 55.879 

Random effects:
 Groups       Name        Variance Std.Dev.
 Neighborhood (Intercept) 0.3663   0.6052  
Number of obs: 295848, groups:  Neighborhood, 84

Fixed effects:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -9.326e+00  6.474e-02 -144.05   <2e-16 ***
Temperature  5.882e-03  9.536e-05   61.68   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
Temperature -0.083
convergence code: 0
Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?


In [11]:
coef(mixedEffectsModel)

$Neighborhood
                                 (Intercept) Temperature
ARMATAGE                          -10.388802 0.005881935
AUDUBON PARK                       -9.681986 0.005881935
BANCROFT                           -9.563530 0.005881935
BELTRAMI                           -9.281869 0.005881935
BOTTINEAU                          -9.160163 0.005881935
BRYANT                             -9.648022 0.005881935
BRYN - MAWR                        -9.624598 0.005881935
CEDAR - ISLES - DEAN               -9.429876 0.005881935
CEDAR RIVERSIDE                    -9.395127 0.005881935
CENTRAL                            -9.311194 0.005881935
CLEVELAND                          -9.166675 0.005881935
COLUMBIA PARK                      -9.619258 0.005881935
COMO                               -9.432673 0.005881935
COOPER                             -9.595045 0.005881935
CORCORAN                           -9.115084 0.005881935
DIAMOND LAKE                      -10.118958 0.005881935
DOWNTOWN EAST    

In [12]:
fixef(mixedEffectsModel)

(Intercept)  Temperature 
-9.325684565  0.005881935

In [13]:
ranef(mixedEffectsModel)

$Neighborhood
                                  (Intercept)
ARMATAGE                         -1.063117808
AUDUBON PARK                     -0.356301055
BANCROFT                         -0.237845438
BELTRAMI                          0.043815920
BOTTINEAU                         0.165522025
BRYANT                           -0.322337565
BRYN - MAWR                      -0.298913824
CEDAR - ISLES - DEAN             -0.104191086
CEDAR RIVERSIDE                  -0.069442631
CENTRAL                           0.014490146
CLEVELAND                         0.159009429
COLUMBIA PARK                    -0.293573726
COMO                             -0.106988841
COOPER                           -0.269360478
CORCORAN                          0.210600960
DIAMOND LAKE                     -0.793273366
DOWNTOWN EAST                     1.003569688
DOWNTOWN WEST                     2.189292548
EAST HARRIET                     -0.468829205
EAST ISLES                        0.422523871
EAST PHILLIPS       

In [14]:
linearMixedEffectsModel <- lmer(Incidents~Temperature + (1 | Neighborhood), data=dailyNeighborhoodIncidents)
summary(linearMixedEffectsModel)

Linear mixed model fit by REML ['lmerMod']
Formula: Incidents ~ Temperature + (1 | Neighborhood)
   Data: dailyNeighborhoodIncidents

REML criterion at convergence: 782468.9

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-7.264 -0.476 -0.244  0.475 35.010 

Random effects:
 Groups       Name        Variance Std.Dev.
 Neighborhood (Intercept) 0.6255   0.7909  
 Residual                 0.8225   0.9069  
Number of obs: 295848, groups:  Neighborhood, 84

Fixed effects:
             Estimate Std. Error t value
(Intercept) 4.568e-01  8.640e-02   5.288
Temperature 3.857e-03  6.901e-05  55.889

Correlation of Fixed Effects:
            (Intr)
Temperature -0.045